In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
289,tt1352824,15000000,8459458,Chloe,Julianne Moore|Liam Neeson|Amanda Seyfried|Max...,Atom Egoyan,"If the one you love was lying to you, how far ...",A doctor hires an escort to seduce her husband...,96,Drama|Thriller|Mystery,"StudioCanal|Montecito Picture Company, The|Out...",3/25/2009,5.9,2009
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,5/21/2008,5.6,2008
113,tt3164256,15000000,3020664,Rock the Kasbah,Bill Murray|Zooey Deschanel|Bruce Willis|Kate ...,Barry Levinson,Opportunity rocks where you least expect it.,A washed-up music producer finds one last shot...,100,Comedy|Music|War,Shangri-La Entertainment|QED International|Dun...,10/22/2015,5.1,2015
447,tt1219342,80000000,140073390,Legend of the Guardians: The Owls of Ga'Hoole,Emily Barclay|Abbie Cornish|Essie Davis|Joel E...,Zack Snyder,On his way to finding a legend...he will becom...,"Soren, a young barn owl, is kidnapped by owls ...",97,Animation|Adventure|Family|Fantasy,Village Roadshow Pictures|Warner Bros.|Animal ...,7/10/2010,6.5,2010
584,tt0253798,24000000,14782676,Out Cold,Jason London|Lee Majors|Zach Galifianakis|Davi...,Brendan Malloy|Emmett Malloy,"They Haven't Quite Figured It All Out, But The...",Animal House meets Casablanca in this outrageo...,89,Comedy,Spyglass Entertainment|Touchstone Pictures,11/21/2001,6.1,2001


In [ ]:
data.describe()

# Предобработка

In [60]:
answers = {} # создадим словарь для ответов

### Функции:
def month(release_date):
  """Выбирает числовое значение месяца из даты формата mm/dd/yyyy, выводит название месяца"""
  month_number = str(release_date).find('/')
  month = int(str(release_date)[:month_number])

  calendar = {
        1: "Январь",
        2: "Февраль",
        3: "Март",
        4: "Апрель",
        5: "Май",
        6: "Июнь",
        7: "Июль",
        8: "Август",
        9: "Сентябрь",
        10: "Октябрь",
        11: "Ноябрь",
        12: "Декабрь"
    }
  return calendar[month]


### тут другие ваши предобработки колонок например:

# добавляется новый столбец с прибылью фильма:
data['profit'] = data['revenue'] - data['budget'] 

# Task 11-12 - учитываем все жанры, потому что есть фильмы, у которых их сразу несколько
new_data_genres = data.copy() # копируем, датафрейм еще раз, чтобы не вносить изменения в копию оригинала
new_data_genres['genres'] = new_data_genres['genres'].str.split('|')
new_data_genres = new_data_genres.explode('genres')

# Task 14 - последовательность действий с режиссерами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_director = data.copy()
new_data_director['director'] = new_data_director['director'].str.split('|')
new_data_director = new_data_director.explode('director')

# Tasks 15-17 - последовательность действий с актерами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_cast = data.copy()
new_data_cast['cast'] = new_data_cast['cast'].str.split('|')
new_data_cast = new_data_cast.explode('cast')

# Task 18,20 - последовательность действий с компаниями точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_company = data.copy()
new_data_company['production_companies'] = new_data_company['production_companies'].str.split('|')
new_data_company = new_data_company.explode('production_companies')

# Task 21-22 - последовательность действий с месяцами точно такая же, как поступали с жанрами из Task 11-12 выше
new_data_month = data.copy()
new_data_month['release_date'] = new_data_month['release_date'].apply(month)
new_data_month = new_data_month.explode('release_date')

# Task 23 - готовим базу одновременно под режиссеров и месяцы по образцам из Tasks 14,21-22 выше
new_data_director_month = new_data_director.copy()
new_data_director_month['release_date'] = new_data_director_month['release_date'].apply(month)
new_data_director_month = new_data_director_month.explode('release_date')

# Task 24 - делаем дополнительный столбец с длиной символов названия каждого фильма, изменяя базу Task 18,20
new_data_title = new_data_company.copy()
new_data_title['title_length'] = new_data_title['original_title'].str.len()

# Task 25 - делаем дополнительный столбец с числом промежутков между словами обзора фильма, изменяя базу Task 18,20
new_data_company_overview = new_data_company.copy()
new_data_company_overview['overview_number'] = new_data_company_overview['overview'].str.count(' ')
new_data_company_overview = new_data_company_overview.explode('overview_number')


#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [ ]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [5]:
# тут пишем ваш код для решения данного вопроса:
data.loc[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011


ВАРИАНТ 2

In [6]:
# можно добавлять разные варианты решения:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011


# 2. Какой из фильмов самый длительный (в минутах)?

In [ ]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [17]:
data.loc[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003


In [8]:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003


# 3. Какой из фильмов самый короткий (в минутах)?





In [ ]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [9]:
data.loc[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011


In [10]:
data[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011


In [11]:
data[data.runtime == data.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

In [12]:
data[data.runtime == data.runtime.min()].imdb_id

768    tt1449283
Name: imdb_id, dtype: object

In [13]:
grouped_data = data.groupby(['original_title'])['runtime'].min().sort_values()
grouped_data

original_title
Winnie the Pooh                                   63
Return to Never Land                              72
The Jungle Book 2                                 72
Home on the Range                                 76
Valiant                                           76
                                                ... 
Pearl Harbor                                     183
King Kong                                        187
Grindhouse                                       191
The Lord of the Rings: The Return of the King    201
Gods and Generals                                214
Name: runtime, Length: 1887, dtype: int64

# 4. Какова средняя длительность фильмов?


In [ ]:
answers['4'] = '2. 110'

In [15]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [ ]:
answers['5'] = '1. 107'

In [16]:
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [ ]:
answers['6'] = '5. Avatar (tt0499549)'

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[['imdb_id', 'original_title']][data['profit'] == data['profit'].max()]

,imdb_id,original_title
239,tt0499549,Avatar


# 7. Какой фильм самый убыточный? 

In [ ]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [20]:
data[['imdb_id', 'original_title']][data['profit'] == data['profit'].min()]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [ ]:
answers['8'] = '1. 1478'

In [22]:
data[data['revenue'] > data['budget']]['imdb_id'].nunique()

1478

In [21]:
data[data['revenue'] > data['budget']].imdb_id.nunique()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [ ]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [8]:
data[data.release_year == 2008].sort_values(by='revenue', ascending=False)[['original_title', 'imdb_id']].iloc[0]

original_title    The Dark Knight
imdb_id                 tt0468569
Name: 599, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [ ]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [11]:
data[data.release_year.isin([2012,2013,2014])].sort_values(by='profit', ascending=True)[['original_title', 'imdb_id']].iloc[0]

original_title    The Lone Ranger
imdb_id                 tt1210819
Name: 1245, dtype: object

# 11. Какого жанра фильмов больше всего?

In [14]:
answers['11'] = '3. Drama'

In [34]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

display(new_data_genres.genres.value_counts().index[0])

'Drama'

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [ ]:
answers['12'] = '1. Drama'

In [35]:
display(new_data_genres[new_data_genres['profit'] > 0].genres.value_counts().index[0])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [ ]:
answers['13'] = '5. Peter Jackson'

In [26]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data.groupby(['director']).revenue.sum().sort_values(ascending=False).index[0])

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [ ]:
answers['14'] = '3. Robert Rodriguez'

In [27]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_director[new_data_director['all_genres'] == 'Action']\
        .groupby('director').imdb_id.nunique().sort_values(ascending=False).index[0])

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [ ]:
answers['15'] = '3. Chris Hemsworth'

In [29]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_cast[new_data_cast['release_year'] == 2012]\
        .groupby('cast').revenue.sum().sort_values(ascending=False).index[0])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [ ]:
answers['16'] = '3. Matt Damon'

In [38]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_cast[new_data_cast['budget'] > new_data_cast['budget'].mean()]\
        .groupby('cast').imdb_id.nunique().sort_values(ascending=False).index[0])

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [ ]:
answers['17'] = '2. Action'

In [37]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_cast[new_data_cast['cast'] == 'Nicolas Cage'].all_genres.value_counts().index[0])

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [ ]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [40]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_company[new_data_company['production_companies'] == 'Paramount Pictures']\
        [['original_title', 'imdb_id', 'profit']].sort_values(by='profit', ascending=True).head(1))

,original_title,imdb_id,profit
925,K-19: The Widowmaker,tt0267626,-64831034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [ ]:
answers['19'] = '5. 2015'

In [41]:
display(data.groupby('release_year').revenue.sum().sort_values(ascending=False).index[0])

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [ ]:
answers['20'] = '1. 2014'

In [42]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_company[new_data_company['production_companies'].str.match('Warner Bros', na=False)]\
        .groupby('release_year').profit.sum().sort_values(ascending=False).index[0])

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [ ]:
answers['21'] = '4. Сентябрь'

In [54]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_month.groupby('release_date').imdb_id.nunique()\
        .sort_values(ascending=False).index[0])

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [ ]:
answers['22'] = '2. 450'

In [55]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_month[new_data_month['release_date']\
                       .isin(['Июнь', 'Июль', 'Август'])].imdb_id.nunique())

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
answers['23'] = '5. Peter Jackson'

In [57]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_director_month[new_data_director_month['release_date']\
        .isin(['Декабрь', 'Январь', 'Февраль'])].groupby('director')\
        .imdb_id.nunique().sort_values(ascending=False).index[0])

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
answers['24'] = '5. Four By Two Productions'

In [59]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_title.groupby('production_companies')['title_length']\
        .mean().sort_values(ascending=False).index[0])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [ ]:
answers['25'] = '3. Midnight Picture Show'

In [61]:
# смотри ПРЕДОБРАБОТКУ датафрейма
display(new_data_company_overview.groupby('production_companies')\
        .overview_number.mean().sort_values(ascending=False).index[0])

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [62]:
display(data[data['vote_average'] > data['vote_average'].quantile(.99)]\
        .sort_values(by='vote_average', ascending=False).original_title)

599                                   The Dark Knight
9                                          Inside Out
34                                               Room
118                                      Interstellar
125                                The Imitation Game
119                           Guardians of the Galaxy
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [ ]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)